Import the necessary libraries and dependencies

In [15]:
from python_lib.modules import *
import torch
import numpy as np
import os
from python_lib.saveasfile import *

# Model initialisation 

Use BlockSave to save all the weights into a `.bin` file

Be sure to `load_state_dict` from a `.pt`/`.pth` file before BlockSave

It's omitted here as I don't have the weights on hand, so I just use a random initialised weights.

In [16]:
model = ECAPA_TDNN(input_size = 2, channels=[8,8,8,8,16], lin_neurons=6, device = "cuda").to("cuda")

BlockSave(model.return_layers(), "fullecapa", "ECAPAweights")

# Generate Input Feats for testing purposes

In [17]:
input_feats = torch.rand([100,2,64]).to("cuda")
input_feats_np = input_feats.cpu().detach().numpy()
input_feats_np = np.reshape(input_feats_np, (100,2,64))
print(input_feats_np)
dim = input_feats_np.shape
print(dim)

flatten_inputs = input_feats_np.flatten()
with open(os.path.join("ECAPAweights", f"ecapainput_2x64_100.bin"), "wb") as f:
            # Write the dimensions down
            f.write(np.array(dim, dtype=np.int32).tobytes())
            # Write the flatten bias down
            f.write(flatten_inputs.tobytes())

[[[0.13175988 0.83884805 0.18142343 ... 0.03749532 0.9003383  0.71930987]
  [0.07325369 0.23795128 0.22634566 ... 0.49048    0.2460568  0.5732361 ]]

 [[0.9554553  0.72399485 0.04407668 ... 0.4900689  0.16904444 0.03300488]
  [0.676814   0.36807913 0.9845918  ... 0.10286719 0.968257   0.07335418]]

 [[0.03088188 0.36878616 0.0039925  ... 0.08633441 0.27036703 0.0772416 ]
  [0.78553414 0.67571807 0.88156545 ... 0.665303   0.21483499 0.41274375]]

 ...

 [[0.731148   0.36628282 0.24018592 ... 0.47873044 0.08777386 0.61594766]
  [0.9290078  0.33120102 0.05099201 ... 0.6845378  0.9484555  0.8847231 ]]

 [[0.8790486  0.74943656 0.5798448  ... 0.57165754 0.8633431  0.40699255]
  [0.80669284 0.44744015 0.24166703 ... 0.2780884  0.9402747  0.19634408]]

 [[0.06906515 0.6387082  0.38524663 ... 0.47731018 0.9629268  0.57335514]
  [0.52897197 0.09806246 0.5458225  ... 0.12157243 0.49805576 0.940619  ]]]
(100, 2, 64)


# Run the model using eval/train mode

In [18]:
model.eval()

model(input_feats)

tensor([[0.1886, 0.1417, 0.1833, 0.1605, 0.1608, 0.1652],
        [0.1889, 0.1416, 0.1829, 0.1611, 0.1604, 0.1651],
        [0.1888, 0.1418, 0.1827, 0.1610, 0.1604, 0.1653],
        [0.1890, 0.1416, 0.1833, 0.1606, 0.1603, 0.1653],
        [0.1888, 0.1414, 0.1832, 0.1605, 0.1606, 0.1655],
        [0.1892, 0.1416, 0.1832, 0.1609, 0.1602, 0.1650],
        [0.1894, 0.1413, 0.1829, 0.1608, 0.1601, 0.1654],
        [0.1892, 0.1415, 0.1833, 0.1610, 0.1600, 0.1650],
        [0.1891, 0.1418, 0.1832, 0.1611, 0.1598, 0.1650],
        [0.1893, 0.1413, 0.1828, 0.1608, 0.1604, 0.1654],
        [0.1890, 0.1415, 0.1832, 0.1607, 0.1603, 0.1652],
        [0.1894, 0.1415, 0.1832, 0.1610, 0.1598, 0.1651],
        [0.1888, 0.1416, 0.1830, 0.1608, 0.1605, 0.1653],
        [0.1893, 0.1413, 0.1829, 0.1609, 0.1602, 0.1653],
        [0.1895, 0.1411, 0.1826, 0.1611, 0.1603, 0.1654],
        [0.1889, 0.1416, 0.1834, 0.1606, 0.1602, 0.1652],
        [0.1893, 0.1415, 0.1833, 0.1609, 0.1599, 0.1652],
        [0.189